In [2]:
import pandas as pd

books = pd.read_csv('books_cleaned.csv')

In [3]:
books['categories'].value_counts()

categories
Fiction                       2111
Juvenile Fiction               390
Biography & Autobiography      311
History                        207
Literary Criticism             124
                              ... 
Human-animal relationships       1
Imperialism                      1
Aged women                       1
Humorous stories                 1
Butlers                          1
Name: count, Length: 479, dtype: int64

In [4]:
category_mapping = {
    'Fiction' : 'Fiction',
    'Juvenile Fiction' : 'Children\'s Fiction',
    'History' : 'Nonfiction',
    'Biography & Autobiography' : 'Nonfiction',
    'Literary Criticism' : 'Nonfiction',
    'Philosophy' : 'Nonfiction',
    'Religion' : 'Nonfiction',
    'Comics & Graphic Novels' : 'Fiction',
    'Drama' : 'Fiction',
    'Juvenile Nonfiction' : 'Children\'s Nonfiction',
    'Science' : 'Nonfiction',
    'Poetry' : 'Fiction'
}

In [5]:
books['simple-categories'] = books['categories'].map(category_mapping)

In [6]:
books[~(books['simple-categories'].isna())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple-categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
8,9780006482079,0006482074,Warhost of Vastmark,Janny Wurts,Fiction,http://books.google.com/books/content?id=uOL0f...,"Tricked once more by his wily half-brother, Ly...",1995.0,4.03,522.0,2966.0,Warhost of Vastmark,9780006482079 Tricked once more by his wily ha...,Fiction
30,9780006646006,000664600X,Ocean Star Express,Mark Haddon;Peter Sutton,Juvenile Fiction,http://books.google.com/books/content?id=I2QZA...,Joe and his parents are enjoying a summer holi...,2002.0,3.50,32.0,1.0,Ocean Star Express,9780006646006 Joe and his parents are enjoying...,Children's Fiction
46,9780007121014,0007121016,Taken at the Flood,Agatha Christie,Fiction,http://books.google.com/books/content?id=3gWlx...,A Few Weeks After Marrying An Attractive Young...,2002.0,3.71,352.0,8852.0,Taken at the Flood,9780007121014 A Few Weeks After Marrying An At...,Fiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5178,9781933648279,1933648279,Night Has a Thousand Eyes,Cornell Woolrich,Fiction,http://books.google.com/books/content?id=3Gk6s...,"""Cornell Woolrich's novels define the essence ...",2007.0,3.77,344.0,680.0,Night Has a Thousand Eyes,"9781933648279 ""Cornell Woolrich's novels defin...",Fiction
5188,9784770028969,4770028962,Coin Locker Babies,村上龍,Fiction,http://books.google.com/books/content?id=87DJw...,Rescued from the lockers in which they were le...,2002.0,3.75,393.0,5560.0,Coin Locker Babies,9784770028969 Rescued from the lockers in whic...,Fiction
5189,9788122200850,8122200850,"Cry, the Peacock",Anita Desai,Fiction,http://books.google.com/books/content?id=_QKwV...,This book is the story of a young girl obsesse...,1980.0,3.22,218.0,134.0,"Cry, the Peacock",9788122200850 This book is the story of a youn...,Fiction
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction


In [7]:
from transformers import pipeline

fiction_categories = ['Fiction', 'Nonfiction']

pipe = pipeline('zero-shot-classification',
                model='facebook/bart-large-mnli',
                device = -1)

Device set to use cpu


In [8]:
sequence = books.loc[books['simple-categories'] == 'Fiction', 'description'].reset_index(drop=True)[0]

In [9]:
pipe(sequence, fiction_categories)

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [10]:
import numpy as np

max_index = np.argmax(pipe(sequence, fiction_categories)['scores'])
max_label = pipe(sequence, fiction_categories)['labels'][max_index]
max_label

'Fiction'

In [11]:
def generate_predictions(sequence, categories):
    predictions = pipe(sequence, categories)
    max_index = np.argmax(predictions['scores'])
    max_label = predictions['labels'][max_index]
    return max_label

In [12]:
from tqdm import tqdm

actual_cats = []
predicted_cats = []

for i in tqdm(range(0, 300)):
    sequence = books.loc[books['simple-categories'] == 'Fiction', 'description'].reset_index(drop=True)[i]
    predicted_cats += [generate_predictions(sequence, fiction_categories)]
    actual_cats += ['Fiction']

  8%|▊         | 23/300 [00:36<07:17,  1.58s/it]


KeyboardInterrupt: 

In [15]:
for i in tqdm(range(0, 300)):
    sequence = books.loc[books['simple-categories'] == 'Nonfiction', 'description'].reset_index(drop=True)[i]
    predicted_cats += [generate_predictions(sequence, fiction_categories)]
    actual_cats += ['Nonfiction']

100%|██████████| 300/300 [07:08<00:00,  1.43s/it]


In [16]:
predicted_df = pd.DataFrame({'actual_categories' : actual_cats, 'predicted_categories' : predicted_cats})

In [1]:
predicted_df

NameError: name 'predicted_df' is not defined

In [18]:
predicted_df['correct_prediction'] = (
    np.where(predicted_df['actual_categories'] == predicted_df['predicted_categories'], 1, 0)
)

In [19]:
predicted_df['correct_prediction'].sum()/len(predicted_df)

np.float64(0.7783333333333333)

In [20]:
isbns = []
predicted_cats = []

missing_cats = books.loc[books['simple-categories'].isna(), ['isbn13', 'description']].reset_index(drop=True)

In [ ]:
for i in tqdm(range(0, len(missing_cats))):
    sequence = missing_cats['description']
    predicted_cats += [generate_predictions(sequence, fiction_categories)]
    isbns += [missing_cats['isbn13'][i]]

 14%|█▍        | 201/1454 [09:59<1:29:44,  4.30s/it]